# Clear GPU RAM


In [ ]:
# from numba import cuda

# # Clear GPU memory
# cuda.select_device(0)
# cuda.close()

# IMPORTS


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import RocCurveDisplay
from sklearn.preprocessing import LabelBinarizer, MinMaxScaler
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import zipfile
import cv2
import os

# DATA LOADING

In [ ]:
!gdown 1Bkvo7XlxG-vXf9bk0OkwAgFpFpkJtDda
!gdown 1T9CJ6hlOK_G6PPAQbMvLvsZK8beP939C
!gdown 1fu4bJA5bn6q_OETgHjqM0jPiG3TelA79
!gdown 1WrMZLYh4QhXhQJwKkAhCjmkUyRFlTf_R

In [ ]:
with zipfile.ZipFile("/content/train_image.zip", "r") as zip_ref:
    zip_ref.extractall("/content/train_images_folder")
with zipfile.ZipFile("/content/train_mask.zip", "r") as zip_ref:
    zip_ref.extractall("/content/train_mask_folder")

with zipfile.ZipFile("/content/test_image.zip", "r") as zip_ref:
    zip_ref.extractall("/content/test_images_folder")
with zipfile.ZipFile("/content/test_mask.zip", "r") as zip_ref:
    zip_ref.extractall("/content/test_mask_folder")

In [ ]:
image_folder = '/content/train_images_folder/image'

# Get a list of image file names in the folder
image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.png')]

# Initialize an empty list to store the image data
image_data = []

# Loop through the image files and convert them to NumPy arrays
for image_file in image_files:
    img = cv2.imread(image_file)
    if img is not None:
        resized_image = cv2.resize(img, (128, 128))
        grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        image_data.append(grayscale_image)

# Convert the list of images to a NumPy array
train_images = np.array(image_data)

#####################################################################################################

image_folder = '/content/train_mask_folder/mask'

# Get a list of image file names in the folder
image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.png')]

# Initialize an empty list to store the image data
image_data = []

# Loop through the image files and convert them to NumPy arrays
for image_file in image_files:
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        resized_image = cv2.resize(img, (128, 128))
        image_data.append(resized_image)

# Convert the list of images to a NumPy array
train_masks = np.array(image_data)

##############################################################################################

image_folder = '/content/test_images_folder/image'

# Get a list of image file names in the folder
image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.png')]

# Initialize an empty list to store the image data
image_data = []

# Loop through the image files and convert them to NumPy arrays
for image_file in image_files:
    img = cv2.imread(image_file)
    if img is not None:
        resized_image = cv2.resize(img, (128, 128))
        grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        image_data.append(grayscale_image)

# Convert the list of images to a NumPy array
test_images = np.array(image_data)

################################################################################################

image_folder = '/content/test_mask_folder/mask'

# Get a list of image file names in the folder
image_files = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder) if filename.endswith('.png')]

# Initialize an empty list to store the image data
image_data = []

# Loop through the image files and convert them to NumPy arrays
for image_file in image_files:
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        resized_image = cv2.resize(img, (128, 128))
        image_data.append(resized_image)

# Convert the list of images to a NumPy array
test_masks = np.array(image_data)

# DATA NORMALIZATION


In [ ]:
train_images = (train_images - np.min(train_images)) / (np.max(train_images) - np.min(train_images))
                                                                                                        # Min Max normalization
train_masks = (train_masks - np.min(train_masks)) / (np.max(train_masks) - np.min(train_masks))

In [ ]:
test_images = (test_images - np.min(test_images)) / (np.max(test_images) - np.min(test_images))
                                                                                                        # Min Max normalization
test_masks = (test_masks - np.min(test_masks)) / (np.max(test_masks) - np.min(test_masks))

In [ ]:
for k in range(len(train_masks)):
  train_masks[k][ train_masks[k] > 0] = 1
for k in range(len(test_masks)):
  test_masks[k][ test_masks[k] > 0] = 1

In [ ]:
train_images, validation_images, train_masks, validation_masks = train_test_split(
                        train_images, train_masks, test_size=0.2, random_state = 42                     # Split
                                                                                    )

In [ ]:
plt.figure(figsize=(10,10))
for i in range(5):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_masks[i], cmap = "Greys")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
for i in range(5):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap = "Greys")
plt.show()

# U-NET


In [ ]:
tf.random.set_seed(
    42
)

In [ ]:
input_layer = tf.keras.layers.Input(shape=(128, 128, 1), name="input_layer")

l1_encode = layers.Conv2D(128, (3, 3), activation='relu', padding="same")(input_layer)
l2_encode = layers.MaxPooling2D((2, 2))(l1_encode)

l3_encode = layers.Conv2D(256, (3, 3), activation='relu', padding="same")(l2_encode)
l4_encode = layers.MaxPooling2D((2, 2))(l3_encode)

l5_encode = layers.Conv2D(256, (3, 3), activation='relu', padding="same")(l4_encode)
l6_encode = layers.MaxPooling2D((2, 2))(l5_encode)

l7_encode = layers.Conv2D(512, (3, 3), activation='relu', padding="same")(l6_encode)

l8_encode = layers.MaxPooling2D((2, 2))(l7_encode)
l9_encode = layers.Conv2D(1024, (3, 3), activation='relu', padding="same")(l8_encode)
l8_decode = layers.Conv2DTranspose(1024, (2, 2), strides=(2, 2), activation='relu')(l9_encode)

l7_decode = layers.Conv2D(512, (3, 3), activation='relu', padding="same")(l8_decode)
l7_decode = layers.Concatenate(axis=-1)([l7_decode, l7_encode]) # skip co
l6_decode = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), activation='relu')(l7_decode)

l5_decode = layers.Conv2D(256, (3, 3), activation='relu', padding="same")(l6_decode)
l5_decode = layers.Concatenate(axis=-1)([l5_decode, l5_encode]) # skip co
l4_decode = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), activation='relu')(l5_decode)

l3_decode = layers.Conv2D(256, (3, 3), activation='relu', padding="same")(l4_decode)
l3_decode = layers.Concatenate(axis=-1)([l3_decode, l3_encode]) # skip co
l2_decode = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), activation='relu')(l3_decode)

l1_decode = layers.Conv2D(128, (3, 3), activation='relu', padding="same")(l2_decode)
l1_decode = layers.Concatenate(axis=-1)([l1_decode, l1_encode]) # skip co

output_layer = layers.Conv2D(1, (1, 1), activation='sigmoid', padding="same")(l1_decode)

In [ ]:
model = tf.keras.Model(input_layer, output_layer)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss ='binary_crossentropy',
              metrics=['accuracy'])

# Segmentation vizualisation

In [ ]:
for k in range(5):

  history = model.fit(train_images, train_masks,batch_size=2, epochs=10,
      validation_data=(validation_images, validation_masks))
  plt.figure(figsize=(10,10))
  pred = model.predict(test_images)
  for i in range(5):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])                    # Displays train predictions
    plt.grid(False)
    plt.imshow(pred[i], cmap="Greys")
  plt.show()

In [ ]:
tf.keras.saving.save_model(
    model,"Unet_3L",overwrite=True, save_format="h5"
)

In [ ]:
def thresholding(thresh, set):
  for i in range(len(set)):
    set[i][set[i] > thresh] = 1
    set[i][set[i] <= thresh] = 0
  return set

In [ ]:
def dice_coefficient(mask1, mask2):
    mask1_flat = mask1.flatten()
    mask2_flat = mask2.flatten()
    intersection = np.sum(mask1_flat * mask2_flat)
    mask1_sum = np.sum(mask1_flat)
    mask2_sum = np.sum(mask2_flat)
    dice = (2.0 * intersection) / (mask1_sum + mask2_sum)
    return dice

In [ ]:
def iou_score(mask1, mask2):
    mask1_flat = mask1.flatten()
    mask2_flat = mask2.flatten()
    intersection = np.sum(mask1_flat * mask2_flat)
    union = np.sum(mask1_flat) + np.sum(mask2_flat) - intersection
    iou = intersection / union
    return iou

In [ ]:
test_pred = model.predict(test_images)

test_pred = thresholding(0, test_pred)

test_DICE=0
test_IOU=0

for j in range(len(test_images)):

    test_DICE = test_DICE + dice_coefficient(test_pred[j] , test_masks[j])/len(test_images)
    test_IOU = test_IOU + iou_score(test_pred[j] , test_masks[j])/len(test_images)

print(test_DICE)
print(test_IOU)

# Train and validation performances during training

In [ ]:
train_DICE = np.zeros(100)
train_IOU = np.zeros(100)

val_DICE = np.zeros(100)
val_IOU = np.zeros(100)

for k in range(100):

  history = model.fit(train_images, train_masks,batch_size=2, epochs=1)

  val_pred = model.predict(validation_images)
  train_pred = model.predict(train_images)

  val_pred = thresholding(0.5, val_pred)
  train_pred = thresholding(0.5, train_pred)

  for j in range(len(train_images)):

    train_DICE[k] = train_DICE[k] + dice_coefficient(train_pred[j] , train_masks[j])/len(train_images)
    train_IOU[k] = train_IOU[k] + iou_score(train_pred[j] , train_masks[j])/len(train_images)

  for j in range(len(validation_images)):

    val_DICE[k] = val_DICE[k] + dice_coefficient(val_pred[j] , validation_masks[j])/len(validation_images)
    val_IOU[k] = val_IOU[k] + iou_score(val_pred[j] , validation_masks[j])/len(validation_images)

plt.subplot(1,2,1)
plt.plot(train_DICE, label='train Dice')
plt.plot(val_DICE, label='validation Dice')
plt.xlabel('Epoch')
plt.ylim([np.min(val_DICE), np.max(train_DICE)])
plt.legend(loc='lower right')

plt.subplot(1,2,2)
plt.plot(train_IOU, label='train IoU')
plt.plot(val_IOU, label='validation IoU')
plt.xlabel('Epoch')
plt.ylim([np.min(val_IOU), np.max(train_IOU)])
plt.legend(loc='lower right')